In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, col, current_timestamp, concat_ws
import psycopg2
import os

# 1. Crée une session Spark
spark = SparkSession.builder \
    .appName("Mastodon Data Processing") \
    .config("spark.jars", "/usr/local/spark/jars/postgresql-42.3.9.jar") \
    .getOrCreate()

# 2. Configure les paramètres de connexion
jdbc_url = "jdbc:postgresql://postgres:5432/" + os.getenv('DB_NAME')
properties = {
    "user": os.getenv('DB_USER'),
    "password": os.getenv('DB_PASSWORD'),
    "driver": "org.postgresql.Driver"
}

# 3. Lit les données depuis PostgreSQL
try:
    bronze_df = spark.read.jdbc(url=jdbc_url, table="bronze_mastodon", properties=properties)
    nombre_lignes_lues_bronze = bronze_df.count()

    # 4. Nettoie la colonne 'content' des balises HTML
    bronze_cleaned_df = bronze_df.withColumn("cleaned_content", regexp_replace(col("content"), "<[^>]+>", ""))

    # 5. Calcule la colonne engagement_count
    bronze_cleaned_df = bronze_cleaned_df.withColumn(
        "engagement_count", 
        col("favourites_count") + col("reblogs_count") + col("replies_count")
    )

    # 6. Filtre les lignes avec certaines conditions
    filtered_df = bronze_cleaned_df.filter(
        (col("favourites_count") > 0) & 
        (col("reblogs_count") > 0) & 
        (col("replies_count") > 0)
    )

    # 7. Filtre les lignes avec une langue non nulle
    filtered_df = filtered_df.filter(col("language").isNotNull())
    nombre_lignes_apres_filtrage = filtered_df.count()

    # 8. Connexion à PostgreSQL via psycopg2
    conn = psycopg2.connect(
        host="postgres",
        database=os.getenv('DB_NAME'),
        user=os.getenv('DB_USER'),
        password=os.getenv('DB_PASSWORD')
    )
    cursor = conn.cursor()

    # 9. Création de la table 'silver_mastodon' si elle n'existe pas déjà
    create_table_query = '''
    CREATE TABLE IF NOT EXISTS silver_mastodon (
        id BIGINT PRIMARY KEY,
        username TEXT NOT NULL,
        cleaned_content TEXT NOT NULL,
        language TEXT NOT NULL,
        favourites_count INT NOT NULL,
        reblogs_count INT NOT NULL,
        replies_count INT NOT NULL,
        engagement_count INT NOT NULL,
        hashtags TEXT,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );
    '''
    cursor.execute(create_table_query)
    conn.commit()

    # 10. Lit les IDs existants dans la table silver_mastodon
    existing_records = spark.read.jdbc(url=jdbc_url, table="silver_mastodon", properties=properties)

    # 11. Crée un DataFrame avec un timestamp pour les nouvelles lignes
    new_records = filtered_df.withColumn("created_at", current_timestamp())

    # 12. Convertit les hashtags en chaîne de texte
    new_records = new_records.withColumn("hashtags", concat_ws(", ", col("hashtags")))

    # 13. Sélectionne uniquement les colonnes nécessaires pour l'insertion
    new_records_to_insert = new_records.select(
        "id",
        "username",
        "cleaned_content",
        "language",
        "favourites_count",
        "reblogs_count",
        "replies_count",
        "engagement_count",
        "hashtags",
        "created_at"
    )

    # Taille du batch pour les transactions
    batch_size = 100
    update_counter = 0
    insert_counter = 0

    # 14. Identifi les enregistrements à mettre à jour
    records_to_update = new_records.join(existing_records, on="id", how="inner")

    nombre_lignes_modifiees_silver = 0
    if records_to_update.count() > 0:
        update_query = '''
            UPDATE silver_mastodon SET 
                username = %s,
                cleaned_content = %s,
                language = %s,
                favourites_count = %s,
                reblogs_count = %s,
                replies_count = %s,
                engagement_count = %s,
                hashtags = %s,
                created_at = CURRENT_TIMESTAMP
            WHERE id = %s;
        '''

        for row in records_to_update.collect():
            cursor.execute(update_query, (
                row.username,
                row.cleaned_content,
                row.language,
                row.favourites_count,
                row.reblogs_count,
                row.replies_count,
                row.engagement_count,
                row.hashtags,
                row.id
            ))
            update_counter += 1

            # Commit toutes les 100 mises à jour
            if update_counter >= batch_size:
                conn.commit()
                print(f"Commit après {update_counter} mises à jour.")
                update_counter = 0

        # Commit final si des mises à jour sont restées non commit
        if update_counter > 0:
            conn.commit()
        nombre_lignes_modifiees_silver = records_to_update.count()

    # 15. Insertion des nouveaux enregistrements
    new_records_to_insert = new_records.alias("new").join(
        existing_records.alias("existing"),
        on="id",
        how="left_anti"
    )
    
    nombre_nouvelles_lignes_ajoutees_silver = new_records_to_insert.count()
    if nombre_nouvelles_lignes_ajoutees_silver > 0:
        for row in new_records_to_insert.collect():
            insert_query = '''
                INSERT INTO silver_mastodon (id, username, cleaned_content, language, favourites_count, reblogs_count, replies_count, engagement_count, hashtags, created_at)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            '''
            cursor.execute(insert_query, (
                row.id,
                row.username,
                row.cleaned_content,
                row.language,
                row.favourites_count,
                row.reblogs_count,
                row.replies_count,
                row.engagement_count,
                row.hashtags,
                row.created_at
            ))
            insert_counter += 1

            # Commit toutes les 100 insertions
            if insert_counter >= batch_size:
                conn.commit()
                print(f"Commit après {insert_counter} insertions.")
                insert_counter = 0

        # Commit final si des insertions sont restées non commit
        if insert_counter > 0:
            conn.commit()

    # 16. Compte le nombre total d'entrées dans la table silver_mastodon
    cursor.execute("SELECT COUNT(*) FROM silver_mastodon;")
    total_lignes_silver = cursor.fetchone()[0]

except Exception as e:
    conn.rollback()  # Annule la transaction en cas d'erreur
    print(f"Une erreur est survenue : {e}")
finally:
    # 17. Ferme la connexion PostgreSQL
    if cursor:
        cursor.close()
    if conn:
        conn.close()

# 18. Ferme la session Spark
spark.stop()

# Affiche les résultats finaux
print(f"Nombre de lignes lues dans 'bronze_mastodon' : {nombre_lignes_lues_bronze}")
print(f"Nombre de lignes modifiées dans 'silver_mastodon' : {nombre_lignes_modifiees_silver}")
print(f"Nombre de nouvelles lignes ajoutées dans 'silver_mastodon' : {nombre_nouvelles_lignes_ajoutees_silver}")
print(f"Nombre total de lignes dans 'silver_mastodon' : {total_lignes_silver}")

Commit après 100 mises à jour.
Commit après 100 mises à jour.
Commit après 100 mises à jour.
Commit après 100 mises à jour.
Commit après 100 mises à jour.
Commit après 100 mises à jour.
Commit après 100 mises à jour.
Commit après 100 mises à jour.
Commit après 100 mises à jour.
Commit après 100 mises à jour.
Commit après 100 mises à jour.
Commit après 100 mises à jour.
Commit après 100 mises à jour.
Commit après 100 mises à jour.
Commit après 100 mises à jour.
Commit après 100 mises à jour.
Commit après 100 mises à jour.
Commit après 100 mises à jour.
Commit après 100 mises à jour.
Commit après 100 mises à jour.
Commit après 100 mises à jour.
Commit après 100 mises à jour.
Commit après 100 mises à jour.
Commit après 100 mises à jour.
Commit après 100 mises à jour.
Commit après 100 insertions.
Commit après 100 insertions.
Nombre de lignes lues dans 'bronze_mastodon' : 3142
Nombre de lignes modifiées dans 'silver_mastodon' : 2513
Nombre de nouvelles lignes ajoutées dans 'silver_mastodon'